In [ ]:
import os
os.chdir("..")

In [ ]:
import h5py
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib
import seaborn as sb
from scipy.stats import gamma

In [ ]:
file = h5py.File("data/fitness.h5")

path = {}
for name,group in file.items():
	shapes = [run["infections"].shape[0] for run in group.values()]
	seedtolook = list(group.keys())[np.argmax(shapes)]
	path[name] = f"{name}/{seedtolook}"

allfitness = {name : file[f"{simulation}/fitness"][:] for name,simulation in path.items()}
infections = {name : file[f"{simulation}/infections"].fields(["t","mut"])[:] for name,simulation in path.items()}
mutation_rates = {name : group.attrs["mutation_rate"] for name,group in file.items()}
fitness_params = {name : {key:val for key,val in group.attrs.items() if "fitness_" in key} for name,group in file.items()}

file.close()

In [ ]:
fitness_params

In [ ]:
nrows = int(np.ceil(len(allfitness)/2))
fig, axs = plt.subplots(nrows, 2, figsize=(12,4*nrows))
axs = axs.flatten()
for name in allfitness.keys():
	i = int(name)
	_,bins,_ = axs[i].hist(allfitness[name]["phi"], bins=50, density=True, alpha=0.5)
	k = fitness_params[name]["fitness_k"]
	theta = fitness_params[name]["fitness_std"] / np.sqrt(k)
	loc = fitness_params[name]["fitness_mean"] - k * theta
	axs[i].plot(bins, gamma.pdf(bins, k, scale=theta, loc=loc))
	axs[i].set_title(f"Mutation rate={mutation_rates[name]}  fitness std={fitness_params[name]['fitness_std']}")

In [ ]:
evolvingfitness = {}
for name,allfit in allfitness.items():
	allfit = pd.DataFrame.from_records(allfit, index="id")
	df = pd.DataFrame.from_records(infections[name], index="mut")
	df["phi"] = allfit.loc[df.index]
	df.reset_index(inplace=True)
	evolvingfitness[name] = df

In [ ]:
f"Mutation rate = {mutation_rates[name]}\nFitness std = {fitness_params[name]['fitness_std']}"

In [ ]:
nrows = len(evolvingfitness)
fig = plt.figure(figsize=(8,4*nrows))
grid = fig.add_gridspec(nrows, 2, width_ratios=(3,1), wspace=0.05, hspace=0.2)
for name,df in evolvingfitness.items():
	i = int(name)
	axbig = fig.add_subplot(grid[i,0])
	sb.histplot(data=df, x="t", y="phi", bins=50, ax=axbig)
	#axbig.set_title(f"Mutation rate={mutation_rates[name]}  fitness std={fitness_params[name]['fitness_std']}")
	xdisplay, ydisplay = axbig.transData.transform((0.1, 0.9))
	axbig.text(0.05, 0.8, f"Mutation rate = {mutation_rates[name]}\nFitness std = {fitness_params[name]['fitness_std']}", transform=axbig.transAxes)
	axbig.set_ylabel("Fitness")
	axbig.set_xlabel("t")
	axbig.set_xlim(left=0)
	axmarg = fig.add_subplot(grid[i,1], sharey=axbig)
	sb.histplot(y=allfitness[name]["phi"], bins=50, stat="density", alpha=0.5, ax=axmarg)
	k = fitness_params[name]["fitness_k"]
	theta = fitness_params[name]["fitness_std"] / np.sqrt(k)
	loc = fitness_params[name]["fitness_mean"] - k * theta
	xs = np.linspace(allfitness[name]["phi"].min(), allfitness[name]["phi"].max(), 1001)
	axmarg.plot(gamma.pdf(xs, k, scale=theta, loc=loc), xs)
	axmarg.tick_params(axis="y", labelleft=False)
	axmarg.set_xlabel("")
	axmarg.set_xticks([])
fig.subplots_adjust()
fig.savefig("analysis/fitness_evolution.png")